### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="l4s-multiple" + fablib.get_bastion_username()

node_conf = [
 {'name': "tx_L4S",    'cores': 8, 'ram': 64, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "tx_legacy",    'cores': 8, 'ram': 64, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "router", 'cores': 8, 'ram': 64, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "delay", 'cores': 8, 'ram': 64, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "rx_L4S",    'cores': 8, 'ram': 64, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "rx_legacy",    'cores': 8, 'ram': 64, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}
]
net_conf = [
 {"name": "net-tx", "subnet": "10.0.0.0/24", "nodes": [{"name": "tx_L4S",   "addr": "10.0.0.100"}, {"name": "tx_legacy",   "addr": "10.0.0.101"}, {"name": "delay", "addr": "10.0.0.2"}]},
 {"name": "net-delay-router", "subnet": "10.0.2.0/24", "nodes": [{"name": "delay",   "addr": "10.0.2.2"}, {"name": "router", "addr": "10.0.2.1"}]},
 {"name": "net-rx", "subnet": "10.0.5.0/24", "nodes": [{"name": "router",   "addr": "10.0.5.1"}, {"name": "rx_L4S", "addr": "10.0.5.100"}, {"name": "rx_legacy", "addr": "10.0.5.101"}]}

]
route_conf = [
 {"addr": "10.0.5.0/24", "gw": "10.0.0.2", "nodes": ["tx_L4S", "tx_legacy"]}, 
 {"addr": "10.0.5.0/24", "gw": "10.0.2.1", "nodes": ["delay"]},

 {"addr": "10.0.0.0/24", "gw": "10.0.5.1", "nodes": ["rx_L4S", "rx_legacy"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.2.2", "nodes": ["router"]}

]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

## We have reserved our slice and now we need to configure our nodes. ##

### Configure Experiment - Prague Kernel Installation

#### Install Prague kernel to all the nodes. Configure DualPI2 on the router. ####

In [ ]:
for node in slice.get_nodes():
    # Download and unzip the kernel package
    node.execute("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.execute("sudo apt install unzip")
    node.execute("unzip l4s-testing.zip")
    
    # Install the kernel packages and update GRUB
    node.execute("sudo dpkg --install debian_build/*")
    node.execute("sudo update-grub")
    node.execute("sudo reboot")

# wait for all nodes to come back up
slice.wait_ssh(progress=True)
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
slice.get_node(name="router").execute("sudo dpkg --install debian_build/*")
slice.get_node(name="router").execute("sudo update-grub")
slice.get_node(name="router").execute("sudo reboot")

In [ ]:
#configuration for DUALPI2 bottleneck on the router
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git && cd iproute2
sudo ./configure
sudo make
sudo make install"""
slice.get_node(name="router").execute(cmd_dualpi2)
slice.get_node(name="router").execute("sudo modprobe sch_dualpi2")

### Configure Experiment - BBRv2 Kernel Installation - Do this if you want to use CUBIC or BBRv2 on the classic senders.

#### Install BBRv2 kernel to classic senders and receivers. ####

In [ ]:
pkg_list = ['linux-headers-5.13.12_5.13.12-2_amd64.deb',
            'linux-libc-dev_5.13.12-2_amd64.deb',
            'linux-image-5.13.12_5.13.12-2_amd64.deb']

cmd_BBRv2 = """sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 5.13.12"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub 
sudo reboot"""


for pkg in pkg_list:
        slice.get_node(name="tx_legacy").execute("wget https://raw.githubusercontent.com/sdatta97/imcbbrrepro/main/setup/{packet}".format(packet=pkg))
        slice.get_node(name="rx_legacy").execute("wget https://raw.githubusercontent.com/sdatta97/imcbbrrepro/main/setup/{packet}".format(packet=pkg))
      
slice.get_node(name="tx_legacy").execute("sudo dpkg -i  *.deb")
slice.get_node(name="tx_legacy").execute(cmd_BBRv2)

slice.get_node(name="rx_legacy").execute("sudo dpkg -i  *.deb")
slice.get_node(name="rx_legacy").execute(cmd_BBRv2)

### Configure Experiment - BBRv3 Kernel Installation - Do this if you want to use BBRv3 on the classic senders.

#### Install BBRv3 kernel to classic senders and receivers. ####

In [ ]:
pkg_list = ['linux-headers-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-image-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-libc-dev_6.4.0-g6e321d1c986a-5_amd64.deb']

cmd_BBRv3 ="""sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 6.4.0"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub
sudo reboot"""

for pkg in pkg_list:
    slice.get_node(name="tx_legacy").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))
    slice.get_node(name="rx_legacy").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))
    
slice.get_node(name="tx_legacy").execute("sudo dpkg -i  *.deb")
slice.get_node(name="tx_legacy").execute(cmd_BBRv3)

slice.get_node(name="rx_legacy").execute("sudo dpkg -i  *.deb")
slice.get_node(name="rx_legacy").execute(cmd_BBRv3)

### Configure Experiment - Install Linux 5.17 to the router - Do this if you want to use L4S-aware FQ-CoDel on the router. Otherwise, use Prague kernel.

In [ ]:
slice.get_node(name="router").execute("sudo wget https://kernel.ubuntu.com/mainline/v5.17/amd64/linux-headers-5.17.0-051700-generic_5.17.0-051700.202203202130_amd64.deb")
slice.get_node(name="router").execute("sudo wget https://kernel.ubuntu.com/mainline/v5.17/amd64/linux-headers-5.17.0-051700_5.17.0-051700.202203202130_all.deb")
slice.get_node(name="router").execute("sudo wget https://kernel.ubuntu.com/mainline/v5.17/amd64/linux-image-unsigned-5.17.0-051700-generic_5.17.0-051700.202203202130_amd64.deb")
slice.get_node(name="router").execute("sudo wget https://kernel.ubuntu.com/mainline/v5.17/amd64/linux-modules-5.17.0-051700-generic_5.17.0-051700.202203202130_amd64.deb")


slice.get_node(name="router").execute("rm -rf new_kernel")
slice.get_node(name="router").execute("mkdir new_kernel")
slice.get_node(name="router").execute("mv *.deb new_kernel/")
# Install the kernel packages and update GRUB

slice.get_node(name="router").execute("sudo dpkg --install new_kernel/*.deb")
slice.get_node(name="router").execute("sudo apt-get -f install -y")

slice.get_node(name="router").execute("sudo update-grub")
slice.get_node(name="router").execute("sudo reboot")

# wait for all nodes to come back up
slice.wait_ssh(progress=True)
slice.get_node(name="router").execute("hostname; uname -a")

#### L4S-aware FQ-CoDel requires new iproute2 version.

In [ ]:
cmd_iproute2 = """wget https://mirrors.edge.kernel.org/pub/linux/utils/net/iproute2/iproute2-6.9.0.tar.gz
tar -xvf iproute2-6.9.0.tar.gz
cd iproute2-6.9.0
sudo apt-get install -y bison
sudo apt-get install -y pkg-config
sudo apt-get install -y flex
make
sudo make install"""

slice.get_node(name="router").execute(cmd_iproute2)
#slice.get_node(name="rx_legacy").execute(cmd_iproute2)

### Check kernel and ip versions of the nodes to be sure. ###

In [ ]:
slice.get_node(name="tx_legacy").execute("sudo ip -V")
slice.get_node(name="rx_legacy").execute("sudo ip -V")

In [ ]:
slice.get_node(name="router").execute("ip -V")

In [ ]:
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### If you want to use BBRv3 with ECN support:

After installing the kernel for BBRv3 as above, you should also update iproute2 on the sender and on the receiver. Open an SSH session on the sender and the receiver (i.e. tx_legacy_node and rx_legacy_node), and apply the following commands. 

In [ ]:
"""
cd /tmp/

wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/0001-ss-output-TCP-BBRv3-diag-information.patch
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/0002-ip-introduce-the-ecn_low-per-route-feature.patch
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/0003-ss-display-ecn_low-if-tcp_info-tcpi_options-TCPI_OPT.patch
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/configure.sh
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/graph_tests.sh
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/median.py
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/nsperf.py
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/run_tests.sh
wget https://github.com/google/bbr/raw/v3/gtests/net/tcp/bbr/nsperf/ss_log_parser.py


sudo mkdir /root/nsperf
sudo mv /tmp/* /root/nsperf/

sudo -i

cd /root/nsperf
chmod +x configure.sh
./configure.sh


cd /root/iproute2/iproute2

git config --global user.name "username"  # put a user name for username
git config --global user.email "email"  # put an email for email


git am /root/nsperf/0001-ss-output-TCP-BBRv3-diag-information.patch
git am /root/nsperf/0002-ip-introduce-the-ecn_low-per-route-feature.patch
git am /root/nsperf/0003-ss-display-ecn_low-if-tcp_info-tcpi_options-TCPI_OPT.patch

cd /root/iproute2/iproute2
make clean
make
sudo make install
"""

# To apply low-latency ecn for a specific route, use the following commands on the terminal.

"""
alias ip=/root/iproute2/iproute2/ip/ip
ip route change ROUTENAME features ecn_low
"""

## We have configured our nodes. Now, we can design and start our experiments. 

### Execute Experiment

In [ ]:
# nodes and instances

tx_L4S_node = slice.get_node(name="tx_L4S")
tx_legacy_node = slice.get_node(name="tx_legacy")
rx_L4S_node = slice.get_node(name="rx_L4S")
rx_legacy_node = slice.get_node(name="rx_legacy")
delay_node = slice.get_node(name="delay")
router_node = slice.get_node(name="router")
# interfaces

tx_L4S_egress_iface  = tx_L4S_node.get_interface(network_name = "net-tx")
tx_legacy_egress_iface  = tx_legacy_node.get_interface(network_name = "net-tx")

delay_ingress_tx_iface  = delay_node.get_interface(network_name = "net-tx")
delay_egress_iface  = delay_node.get_interface(network_name = "net-delay-router")
delay_ingress_tx_name = delay_ingress_tx_iface.get_device_name()
delay_egress_name = delay_egress_iface.get_device_name()

router_ingress_iface  = router_node.get_interface(network_name = "net-delay-router")
router_egress_iface  = router_node.get_interface(network_name = "net-rx")
router_egress_name  = router_egress_iface.get_device_name()


rx_L4S_ingress_iface  = rx_L4S_node.get_interface(network_name = "net-rx")
rx_legacy_ingress_iface  = rx_legacy_node.get_interface(network_name = "net-rx")

In [ ]:
# generate full factorial experiment
import itertools


# exp_factors define our experiment factors. You can arrange it here according to your experiment configuration.


# IMPORTANT THINGS TO CONSIDER:

    # For DualPI2, on the router, you need to use the Prague kernel and configure DualPI2.
    # For FQ_Codel_L4S (L4S-aware FQ-CoDel), on the router, you need to use the Linux kernel version 5.16 and beyond and also update iproute2. In this notebook, we use Linux kernel 5.17 and iproute2-6.9.0.
    # On L4S senders, you can use Prague and L4S-compatible BBRv2 with Prague kernel with AccECN enabled.
    # On classic senders, you can use BBRv2 with the BBRv2 kernel (5.13.12) we installed in this notebook. You can use BBRv3 with BBRv3 kernel (6.4.0+) that we installed in this notebook. You can use CUBIC and BBRv1 with both kernels (in bbrv3 kernel, bbrv1's name is bbr1). We used 5.13.12 for bbrv1 and cubic. 

    # Therefore, please be aware of which kernel is installed on which node before starting the experiments and select AQM type and TCP CC type according to those installed kernels. 

    # ecn_threshold parameter will not be applied to FIFO, pie_drop, and Codel_drop. They are drop-based AQMs.

    # ecn_fallback parameter is for TCP Prague specifically to enable or disable its fallback algorithm for single queue classic ECN queues. It is disabled by default but you can enable it.

    # In a single run, for instance, if you run [prague vs cubic] and [L4S bbr2 vs cubic], on the classic sender, the same file name will be generated and it will cause problem. 
    # Therefore, DO NOT RUN multiple CCA combinations in a single run.
    # Namely, for instance, run for Prague vs CUBIC, and save the data. Then, run for L4S-compatible BBRv2 vs CUBIC and save the data. 
    # DONT RUN Prague vs CUBIC and L4S-compatible BBRv2 vs CUBIC in a single run and then save the data, this will cause problem because CUBIC file names will be the same for both cases.
    # You can configure the name_tx_L4S and name_tx_legacy parameters in the next cell to fix this, if you want. 

    # You can use multiple notebooks for different CCA combinations.

exp_factors = {
    'n_bdp': [0.5,1,2,4,8],  # n x bandwidth delay product (BDP)
    'btl_capacity': [100], #in Mbps 
    'base_rtt': [10], # in ms 
    'aqm': ['FIFO', 'single_queue_FQ', 'pie_drop', 'Codel_drop', 'Codel', 'FQ', 'FQ_Codel', 'FQ_Codel_L4S', 'DualPI2'],
    'ecn_threshold': [5], # in ms
    'ecn_fallback': [0],  #fallback algorithm, TCP Prague falls back to classic TCP when it detects single queue classic ECN bottleneck # 0: OFF, 1: ON  #'ecn_fallback': [0, 1]
    'tx_L4S_ecn': [3],  # 0: noecn, 1: ecn, 3: accecn #'rx_L4S_ecn': [0, 1, 3]
    'tx_legacy_ecn': [0],  # 0: noecn, 1: ecn #'rx_legacy_ecn': [0, 1]
    'rx_L4S_ecn': [3],  # 0: noecn, 1: ecn, 3: accecn #'rx_L4S_ecn': [0, 1, 3]
    'rx_legacy_ecn': [0],  # 0: noecn, 1: ecn #'rx_legacy_ecn': [0, 1]
    'cc_tx_L4S': ["prague"], # prague, bbr2
    'cc_tx_legacy': ["cubic"], # cubic, bbr, bbr2, bbr3
    'N_L4S':[1, 2, 5, 10, 25],
    'N_legacy':[1, 2, 5, 10, 25],
    'trial': [1,2,3,4,5,6,7,8,9,10] # it means total 10 trials here.
}

factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []

seen_combinations = set()

# Removing ECN factor from FIFO bottleneck because it does not support ECN. This could be done also for pie_drop and codel_drop but we have not done.
# Removing the cases where ECN Threshold is less than or equal to the buffer size in time, these cases are not meaningful in practice.

for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    if temp_dict['n_bdp'] * temp_dict['base_rtt'] >= temp_dict['ecn_threshold']:
        if temp_dict['aqm'] == 'FIFO':
            del temp_dict['ecn_threshold']
        # Convert dict to a frozenset for set operations
        fs = frozenset(temp_dict.items())
    
        if fs not in seen_combinations:
            seen_combinations.add(fs)
            exp_lists.append(temp_dict)

data_dir_tx_L4S = slice_name + 'singlebottleneck'+"-tx_L4S"
data_dir_tx_legacy = slice_name + 'singlebottleneck'+"-tx_legacy"

print("Number of experiments:",len(exp_lists))

In [ ]:
# IMPORTANT THINGS TO CONSIDER:

    # If you use BBRv2 on the classic sender with ECN, check the code below carefully. There are some additional instructions to enable ECN. 

# run experiments
import time
d = 60 #duration in seconds

em = [delay_ingress_tx_name, delay_egress_name]

for exp in exp_lists:

    # check if we already ran this experiment
    # (allow stop/resume)
    name_tx_L4S="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d_%d_%d" % (exp['cc_tx_L4S'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['N_L4S'], exp['N_legacy'], exp['trial'])
    name_tx_legacy="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d_%d_%d" % (exp['cc_tx_legacy'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['N_L4S'], exp['N_legacy'], exp['trial'])

    file_out_tx_L4S_json = name_tx_L4S+"-result.json"
    stdout_tx_L4S_json, stderr_tx_L4S_json = tx_L4S_node.execute("ls " + file_out_tx_L4S_json, quiet=True) 
    
    file_out_tx_legacy_json =name_tx_legacy+"-result.json"
    stdout_tx_legacy_json, stderr_tx_legacy_json = tx_legacy_node.execute("ls " + file_out_tx_legacy_json, quiet=True) 
    

    if len(stdout_tx_L4S_json) and len(stdout_tx_legacy_json):
        print("Already have " + name_tx_L4S + " and "+ name_tx_legacy + ", skipping")

    elif len(stderr_tx_L4S_json) or len(stderr_tx_legacy_json):
        print("Running experiment to generate " + name_tx_L4S + " and "+ name_tx_legacy) 

        cc_tx_legacy = exp['cc_tx_legacy']

        if cc_tx_legacy == 'bbr3': # we are doing this, because in BBRv3 kernel, name of BBRv3 is "bbr" not "bbr3". 

            cc_tx_legacy = "bbr"

        tx_L4S_node.execute("sudo modprobe tcp_" + exp['cc_tx_L4S'])
        tx_L4S_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc_tx_L4S'])
        tx_L4S_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['tx_L4S_ecn'])) 
        
        tx_legacy_node.execute("sudo modprobe tcp_" + cc_tx_legacy)
        tx_legacy_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + cc_tx_legacy)
        tx_legacy_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['tx_legacy_ecn'])) 

        rx_legacy_node.execute("sudo modprobe tcp_" + exp['cc_tx_legacy'])
        rx_legacy_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + cc_tx_legacy)
        rx_legacy_node.execute("sudo sysctl -w net.ipv4.tcp_ecn={ecn_type}".format(ecn_type=exp['rx_legacy_ecn']))      

        
        # If you use BBRv2 on the classic sender with ECN, to enable ECN, you need to uncomment the following two lines. 
        
        #tx_legacy_node.execute("echo 1 | sudo tee /sys/module/tcp_bbr2/parameters/ecn_enable") 
        #tx_legacy_node.execute("echo 0 | sudo tee /sys/module/tcp_bbr2/parameters/ecn_max_rtt_us") 

        
        # If you use BBRv2 on the classic sender and receiver with ECN, to enable DCTCP-style marking on the receiver side, you need to uncomment the following two lines. 
        # Otherwise, they will do classic marking, instead of DCTCP-style marking.
        
        #rx_legacy_node.execute("echo 1 | sudo tee /sys/module/tcp_bbr2/parameters/ecn_enable") 
        #rx_legacy_node.execute("echo 0 | sudo tee /sys/module/tcp_bbr2/parameters/ecn_max_rtt_us")
        
        # delay at emulator
        for e in em:
            cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=exp['base_rtt']/2)
            delay_node.execute(cmds)
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1
        
        
        #ecn-fallback configuration

        if exp['cc_tx_L4S']=='prague':
            commands = "echo {value} | sudo tee /sys/module/tcp_prague/parameters/prague_ecn_fallback".format(value=str(exp['ecn_fallback']))
            tx_L4S_node.execute(commands)
        
        #receiver ecn configuration
        
        rx_L4S_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc_tx_L4S'])
        rx_L4S_node.execute("sudo sysctl -w net.ipv4.tcp_ecn={ecn_type}".format(ecn_type=exp['rx_L4S_ecn']))
         
        #aqm type selection
        cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], buffer=btl_limit)
        
        cmds_specific_initial = "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: ".format(iface=router_egress_name)
        
        cmds_specific = {
        'FIFO': "bfifo limit {buffer}".format(buffer=btl_limit),
        'single_queue_FQ': "fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'Codel': "codel limit {packet_limit} target {target}ms interval 100ms ecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'Codel_drop': "codel limit {packet_limit} target {target}ms interval 100ms noecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'FQ': "fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'FQ_Codel': "fq_codel limit {packet_limit} target {target}ms interval 100ms ecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'FQ_Codel_L4S': "fq_codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold 1ms ce_threshold_selector 0x01/0x01".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'pie_drop': "pie limit {packet_limit} target {target}ms noecn".format(packet_limit=packet_number, target=exp.get('ecn_threshold', 0)),
        'DualPI2': "dualpi2 target {threshold}ms".format(threshold=exp.get('ecn_threshold', 0))
        }

        cmds_aqm = {key: cmds_specific_initial + cmd for key, cmd in cmds_specific.items()}
        
        router_node.execute(cmds_prefix)
        router_node.execute(cmds_aqm[ exp['aqm'] ])
            
        rx_L4S_node.execute("killall iperf3")
        rx_legacy_node.execute("killall iperf3")

        # Example SS script, if you want, you can uncomment corresponding lines but we have not used ss output in these experiments.
        
        #ss_tx_L4S_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.5.100 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"
        #ss_tx_legacy_script="rm -f {flow}-ss.txt; start_time=$(date +%s); while true; do ss --no-header -eipn dst 10.0.5.101 | ts '%.s' | tee -a {flow}-ss.txt; current_time=$(date +%s); elapsed_time=$((current_time - start_time));  if [ $elapsed_time -ge {duration} ]; then break; fi; sleep 0.1; done;"
                
        rx_L4S_node.execute("iperf3 -s -1 -p 4000 -D")
        rx_legacy_node.execute("iperf3 -s -1 -p 5000 -D")

        #tx_L4S_node.execute_thread(ss_tx_L4S_script.format(flow=name_tx_L4S, duration=d))
        #tx_legacy_node.execute_thread(ss_tx_legacy_script.format(flow=name_tx_legacy, duration=d))
        
        tx_L4S_node.execute_thread("sleep 1; iperf3 -c 10.0.5.100 -t {duration} -P {flows} -C {cc} -p 4000 -J > {flow}-result.json".format(flow =name_tx_L4S, duration=d, flows=exp['N_L4S'], cc=exp['cc_tx_L4S']))
        stdout, stderr = tx_legacy_node.execute("sleep 1; iperf3 -c 10.0.5.101 -t {duration} -P {flows} -C {cc} -p 5000 -J > {flow}-result.json".format(flow =name_tx_legacy, duration=d, flows=exp['N_legacy'], cc=exp['cc_tx_legacy']))
        time.sleep(3)  # time.sleep(1)
        
print("finished")

## Our experiments are finished. Now, we can process the data and generate the required files.

### Analysis of the results

In [ ]:
cmds_py_install = '''
            sudo apt -y install python3-pip
            pip install numpy
            pip install matplotlib
            pip install pandas
            '''

tx_L4S_node.execute(cmds_py_install)
tx_legacy_node.execute(cmds_py_install)

#### ANALYSIS WITHOUT SS OUTPUT ####

In [ ]:
tx_legacy_node.execute('mkdir '+data_dir_tx_legacy)
tx_legacy_node.execute('mv *.json '+ data_dir_tx_legacy)

tx_L4S_node.execute('mkdir '+data_dir_tx_L4S)
tx_L4S_node.execute('mv *.json '+ data_dir_tx_L4S)

In [ ]:
tx_L4S_node.upload_file("/home/fabric/work/L4SProject/analysis.py", f"/home/ubuntu/{data_dir_tx_L4S}/analysis.py")
tx_legacy_node.upload_file("/home/fabric/work/L4SProject/analysis.py", f"/home/ubuntu/{data_dir_tx_legacy}/analysis.py")

In [ ]:
tx_L4S_node.execute(f'python3 /home/ubuntu/{data_dir_tx_L4S}/analysis.py')
tx_legacy_node.execute(f'python3 /home/ubuntu/{data_dir_tx_legacy}/analysis.py')

In [ ]:
tx_L4S_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/tput_tx_L4S.json",f"/home/ubuntu/{data_dir_tx_L4S}/throughput_data.json")
tx_L4S_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/srtt_tx_L4S.json",f"/home/ubuntu/{data_dir_tx_L4S}/srtt_data.json")

tx_legacy_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/tput_tx_legacy.json",f"/home/ubuntu/{data_dir_tx_legacy}/throughput_data.json")
tx_legacy_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/srtt_tx_legacy.json",f"/home/ubuntu/{data_dir_tx_legacy}/srtt_data.json")

In [ ]:
import json
import os

# Initialize empty variables
throughput_data = {}
srtt_data = {}

# Directory containing JSON files
data_directory = '/home/fabric/work/L4SProject/ScalableBBRv2/results/cubic_noecn/'

# List of JSON files in the directory
json_files = [f for f in os.listdir(data_directory) if f.endswith('.json')]

# Load data from each JSON file and update the variables
for file_name in json_files:
    file_path = os.path.join(data_directory, file_name)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Check if the file contains throughput data or srtt data based on its name
    if 'tput' in file_name:
        throughput_data.update(data)
    elif 'srtt' in file_name:
        srtt_data.update(data)

### Now we have json files for both tput and srtt. We will use these files in plotting notebooks. However, be careful about the comment below to use our plotting notebook.

In our data processing notebook, to identify and differentiate the flows, JSON file name should start with classic TCP CC type and it should include srtt or tput.
For instance: for experiments Prague vs CUBIC, cubic_tput_tx_L4S.json contains all the throughput data and in those experiments the classic flow is cubic.
For instance: for experiments Prague vs BBRv2, bbr2_tput_tx_L4S.json contains all the throughput data and in those experiments the classic flow is bbrv2.


Therefore, you should arrange the file names accordingly.

## Extra: For SS output - By default it is not enabled in this notebook. You can skip this section, if you did not use SS in your experiments.

In [ ]:
# Function generating a script to create the csv files
def generate_script(name): 
    return """
        rm -f {name}-ss.csv
        cat {name}-ss.txt | sed -e ":a; /<->$/ {{ N; s/<->\\n//; ba; }}"  | grep "iperf3" | grep -v "SYN-SENT" > {name}-ss-processed.txt 
        cat {name}-ss-processed.txt | awk '{{print $1}}' > ts-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP 'cwnd:\\d+' | awk -F ':' '{{print $2}}' > cwnd-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP 'rtt:.*?(\\s|$)' | awk -F '[:,]' '{{print $2}}' | tr -d ' '  | cut -d '/' -f 1 > srtt-{name}.txt 
        cat {name}-ss-processed.txt | grep -oP 'fd=.*?(\\s|$)' | awk -F '[=,]' '{{print $2}}' | tr -d ')' | tr -d ' '   > fd-{name}.txt
        paste ts-{name}.txt fd-{name}.txt cwnd-{name}.txt srtt-{name}.txt -d ',' > {name}-ss.csv
    """.format(name=name)


In [ ]:
for exp in exp_lists:
    
    name_tx_L4S="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx_L4S'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['trial'])
    name_tx_legacy="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx_legacy'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['trial'])
    
    
    file_out_tx0_csv = name_tx_L4S+"-ss.csv"
    stdout_tx0_csv, stderr_tx0_csv = tx_L4S_node.execute("ls " + file_out_tx0_csv, quiet=True) 
    
    file_out_tx1_csv = name_tx_L4S+"-ss.csv"
    stdout_tx1_csv, stderr_tx1_csv = tx_legacy_node.execute("ls " + file_out_tx1_csv, quiet=True) 

    if len(stdout_tx0_csv) and len(stdout_tx1_csv):
        print("Already have " + name_tx_L4S + " and "+ name_tx_legacy + ", skipping")

    elif len(stderr_tx0_csv) or len(stderr_tx1_csv):
        print("Running to generate csv files " + name_tx_L4S + " and "+ name_tx_legacy)
    
        script_tx0 = generate_script(name_tx_L4S)
        script_tx1 = generate_script(name_tx_legacy)

        tx_L4S_node.execute(script_tx0)
        tx_legacy_node.execute(script_tx1)

In [ ]:
tx_legacy_node.execute('mkdir '+data_dir_tx_legacy)

tx_legacy_node.execute('mv *.json '+ data_dir_tx_legacy)
tx_legacy_node.execute('mv *.csv '+ data_dir_tx_legacy)
tx_legacy_node.execute('mv *.txt '+ data_dir_tx_legacy)

tx_L4S_node.execute('mkdir '+data_dir_tx_L4S)

tx_L4S_node.execute('mv *.json '+ data_dir_tx_L4S)
tx_L4S_node.execute('mv *.csv '+ data_dir_tx_L4S)
tx_L4S_node.execute('mv *.txt '+ data_dir_tx_L4S)

In [ ]:
tx_legacy_node.upload_file("/home/fabric/work/L4SProject/analysis_with_ss.py", f"/home/ubuntu/{data_dir_tx_legacy}/analysis_with_ss.py")
tx_L4S_node.upload_file("/home/fabric/work/L4SProject/analysis_with_ss.py", f"/home/ubuntu/{data_dir_tx_L4S}/analysis_with_ss.py")

In [ ]:
tx_legacy_node.execute(f"chmod +x /home/ubuntu/{data_dir_tx_legacy}/analysis_with_ss.py")
tx_L4S_node.execute(f"chmod +x /home/ubuntu/{data_dir_tx_L4S}/analysis_with_ss.py")

In [ ]:
tx_legacy_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/tput_tx_legacy.json",f"/home/ubuntu/{data_dir_tx_legacy}/throughput_data.json")
tx_legacy_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/srtt_tx_legacy.json",f"/home/ubuntu/{data_dir_tx_legacy}/srtt_data.json")

In [ ]:
tx_legacy_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/cwnd_tx1.csv",f"/home/ubuntu/{data_dir_tx_legacy}/consolidated_cwnd_data.csv")
tx_legacy_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/time_tx1.csv",f"/home/ubuntu/{data_dir_tx_legacy}/consolidated_time_data.csv")
tx_legacy_node.download_file("/home/fabric/work/L4SProject/ScalableBBRv2/srtt_tx1.csv",f"/home/ubuntu/{data_dir_tx_legacy}/srtt_all.csv")

### At this point, we have cwnd, srtt, and timestamp values as time series for each experiment. While you can plot them if desired, it is beyond the scope of this study.